# Graph Convoultional Network

In [1]:
import math

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as L
import tensorflow.keras.metrics as metrics
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: tf.config.experimental.set_memory_growth(gpus[0], True)

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
class GraphConvolution(tf.keras.layers.Layer):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, out_features, use_bias=True):
        super(GraphConvolution, self).__init__()
        
        self.out_features = out_features        
        self.use_bias = use_bias
    
    def build(self, input_shape):
        
        stddv = 1 / math.sqrt(self.out_features)
        initializer = tf.keras.initializers.RandomUniform(
            minval=-stddv,
            maxval=stddv
        )
        
        self.kernel = self.add_variable(
            name="kernel", 
            shape=[int(input_shape[-1]), self.out_features],
            initializer=initializer
        )
        
        if self.use_bias:
            self.bias = self.add_variable(
                name="bias",
                shape=[self.out_features],
                initializer=initializer
            )
    def call(self, input, adj):
        support = tf.matmul(input, self.kernel)
        output = tf.sparse.sparse_dense_matmul(adj, support)
        if self.bias:
            return output + self.bias
        else:
            return output

    def __repr__(self):

    
        return self.__class__.__name__ + ' (' \
               + "output features is : " \
               + str(self.out_features) + ')'

layer = GraphConvolution(10)
layer

GraphConvolution (output features is : 10)

In [3]:
layer = GraphConvolution(5)

In [4]:
class GCN(tf.keras.Model):
    def __init__(self, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nhid)
        self.gc2 = GraphConvolution(nclass)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def forward(self, x, adj, training=False):
        x = tf.relu(self.gc1(x, adj))
        x = self.dropout(x, self.dropout, training=training)
        x = self.gc2(x, adj)
        return tf.nn.log_softmax(x, axis=1)

In [5]:
model = GCN(50, 2, 0.3)

In [7]:
model